In [1]:
import os
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import torch

In [2]:
is_kaggle = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
path = Path('../input/visual-taxonomy') if is_kaggle else Path('data')

if is_kaggle:
    utils_path = Path('../input/attribute-extraction-meesho')
    from shutil import copyfile
    copyfile(src=utils_path/'data_preparation.py', dst='../working/data_preparation.py')

In [3]:
from src.data_preparation import *

In [ ]:
cat_info = pd.read_parquet(path/'category_attributes.parquet')
df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')

In [ ]:
from transformers import BlipImageProcessor, BertTokenizerFast

ckpt = "Salesforce/blip-itm-base-coco"
img_processor = BlipImageProcessor.from_pretrained(ckpt)
tokenizer = BertTokenizerFast.from_pretrained(ckpt)

In [ ]:
df = process_df(df, path)
cat_info = process_cat_info(cat_info, tokenizer)

In [ ]:
batch_size = 4
dl = MeeshoDataloader(df, cat_info, batch_size=batch_size, img_processor=img_processor)